In [ ]:
https://github.com/abyanjan/Fine-Tune-BERT-for-Text-Classification/blob/master/Fine_tune_bert_with_hugging%20face.ipynb

In [ ]:
pip install tensorflow transformers pandas scikit-learn


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Data.csv", names=["text", "intent"])


In [ ]:
label_encoder = LabelEncoder()
df['intent_label'] = label_encoder.fit_transform(df['intent'])


In [ ]:
df['intent_label']

0      2
1      1
2      1
3      1
4      1
      ..
396    0
397    0
398    0
399    0
400    0
Name: intent_label, Length: 401, dtype: int64

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X = tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors='tf')



In [ ]:
print(X['input_ids'].shape)


(401, 23)


In [ ]:
# Access the 'input_ids' key from the tokenized input
X_input_ids = X['input_ids']

# Convert tensor to numpy array for train_test_split
X_input_ids_numpy = X_input_ids.numpy()

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_input_ids_numpy, df['intent_label'].tolist(), test_size=0.2, random_state=42)




In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
# Assuming 'input_ids' is the key for tokenized input
X_train = {'input_ids': X_train}
X_val = {'input_ids': X_val}

# Convert y_train and y_val to NumPy arrays
y_train = np.array(y_train)
y_val = np.array(y_val)

# Now, you can fit the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=3, batch_size=32)



Epoch 1/3
10/10 [==============================] - 49s 578ms/step - loss: 0.8463 - accuracy: 0.6969 - val_loss: 0.5985 - val_accuracy: 0.9259
Epoch 2/3
10/10 [==============================] - 2s 226ms/step - loss: 0.4381 - accuracy: 0.9406 - val_loss: 0.2832 - val_accuracy: 0.9753
Epoch 3/3
10/10 [==============================] - 2s 233ms/step - loss: 0.2280 - accuracy: 0.9719 - val_loss: 0.1804 - val_accuracy: 0.9630


In [ ]:
# Save the entire model to a file
model.save('bert_intent_model')

# If you want to save only the model weights, you can use:
# model.save_weights('bert_intent_model_weights.h5')


In [ ]:
import numpy as np
from transformers import BertTokenizer
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('bert_intent_model')

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# New sentence for prediction
new_sentence = "How do animals communicate?"

# Tokenize the new sentence
tokenized_input = tokenizer(new_sentence, padding=True, truncation=True, return_tensors='tf')

# Extract input_ids, attention_mask, and token_type_ids from tokenized_input
input_ids = tokenized_input['input_ids']
attention_mask = tokenized_input['attention_mask']
token_type_ids = tokenized_input['token_type_ids']

# Make predictions
predictions = loaded_model.predict({'input_ids': input_ids, 'attention_mask': attention_mask, 'token_type_ids': token_type_ids})
print(predictions)
# Ensure predictions is an array (convert to numpy array if needed)
predictions = np.array(predictions)
print(predictions)
# Find the argmax without specifying axis for one-dimensional array
predicted_label = np.argmax(predictions)
print(predicted_label)
# Map numerical label back to original intent label
predicted_intent = label_encoder.inverse_transform([predicted_label])[0]

# Print the predicted intent for the sentence
print(f"Sentence: {new_sentence}\nPredicted Intent: {predicted_intent}")


1/1 [==============================] - 1s 1s/step
{'logits': array([[-0.4549155 ,  1.7094026 , -0.64164025]], dtype=float32)}
{'logits': array([[-0.4549155 ,  1.7094026 , -0.64164025]], dtype=float32)}
0
Sentence: How do animals communicate?
Predicted Intent: General query
